In [23]:
from geopy.geocoders import Nominatim
import folium
import pandas as pd
import requests
from pandas.io.json import json_normalize

In [24]:
ClIENT_ID = '5J5PBAMUUA22S2KM2USBTRLCEWPHIKO0P05HOZA5KEGKLWDZ' # your Foursquare ID
ClIENT_SECRET = 'NKOKFXFBVPRWAUUSSR0F4UVELLHYX4YVALN1HRJHLMUP45LK' # your Foursquare Secret
VERSION = '20180604'
LIMIT =30
print('Your credentails:')
print('Foursquare_ID: ' + ClIENT_ID)
print('Foursquare_Secret:' + ClIENT_SECRET)

Your credentails:
Foursquare_ID: 5J5PBAMUUA22S2KM2USBTRLCEWPHIKO0P05HOZA5KEGKLWDZ
Foursquare_Secret:NKOKFXFBVPRWAUUSSR0F4UVELLHYX4YVALN1HRJHLMUP45LK


In [25]:
city = 'Mumbai'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude) #printing the latitude and longitude of mumbai

18.9387711 72.8353355


In [32]:
search_query = 'Restaurant'
radius = 10000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url #creatinng the url that we need 

'https://api.foursquare.com/v2/venues/search?client_id=5J5PBAMUUA22S2KM2USBTRLCEWPHIKO0P05HOZA5KEGKLWDZ&client_secret=NKOKFXFBVPRWAUUSSR0F4UVELLHYX4YVALN1HRJHLMUP45LK&ll=18.9387711,72.8353355&v=20180604&query=Restaurant&radius=10000&limit=30'

In [33]:
results = requests.get(url).json()

Restaurants in Mumbai

In [34]:
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe.head() #creating a datafframe that has all the details of the restaurants

C:\Users\deepa\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode
0,4e4c0a4122713bd908c641ff,Farhang Restaurant,"[{'id': '4bf58dd8d48988d1cc941735', 'name': 'S...",v-1588314712,False,Opposite GPO,Ballard Estate,18.938165,72.837917,"[{'label': 'display', 'lat': 18.93816453050136...",280,IN,Mumbai,Mahārāshtra,India,"[Opposite GPO (Ballard Estate), Mumbai, Mahārā...",NaN
1,5263f0f5498edc1ad5f3270f,Nanumal Bhojraj Restaurant - Fort,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1588314712,False,"Near GPO, Fort, VT",S. Bhagat Singh Road,18.937974,72.837663,"[{'label': 'display', 'lat': 18.93797448587734...",260,IN,Mumbai,Mahārāshtra,India,"[Near GPO, Fort, VT (S. Bhagat Singh Road), Mu...",400001
2,4fb5276be4b0b34b9ac27a1b,Iran Like Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1588314712,False,"Palton Road, Near Haj House",NaN,18.945343,72.836809,"[{'label': 'display', 'lat': 18.94534293649397...",747,IN,Mumbai,Mahārāshtra,India,"[Palton Road, Near Haj House, Mumbai 400001, M...",400001
3,4d9f2bfe9b91a1cda6c365c0,New Majestic Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1588314712,False,Opp CST,NaN,18.938972,72.835517,"[{'label': 'display', 'lat': 18.93897156089674...",29,IN,Mumbai,Mahārāshtra,India,"[Opp CST, Mumbai, Mahārāshtra, India]",NaN
4,4c09449eed2595217aee1256,Stadium Restaurant & Stores,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1588314712,False,Veer Nariman Road,"Next to Vodafone Store, Churchgate",18.933173,72.826929,"[{'label': 'display', 'lat': 18.93317339191467...",1082,IN,Mumbai,Mahārāshtra,India,"[Veer Nariman Road (Next to Vodafone Store, Ch...",400023


In [35]:
dataframe.isnull().sum() #checking for null values in the data

id                            0
name                          0
categories                    0
referralId                    0
hasPerk                       0
location.address             13
location.crossStreet         20
location.lat                  0
location.lng                  0
location.labeledLatLngs       0
location.distance             0
location.cc                   0
location.city                 9
location.state                9
location.country              0
location.formattedAddress     0
location.postalCode          20
dtype: int64

In [37]:
# keep only columns that include venue name, and anything that is associated with location
Restaurant_clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_dataframe = dataframe.loc[:,Restaurant_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

# filter the category for each row
clean_Restaurant_dataframe['categories'] = clean_Restaurant_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Restaurant_dataframe.columns = [column.split('.')[-1] for column in clean_Restaurant_dataframe.columns]

clean_Restaurant_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,id
0,Farhang Restaurant,Steakhouse,Opposite GPO,Ballard Estate,18.938165,72.837917,"[{'label': 'display', 'lat': 18.93816453050136...",280,IN,Mumbai,Mahārāshtra,India,"[Opposite GPO (Ballard Estate), Mumbai, Mahārā...",NaN,4e4c0a4122713bd908c641ff
1,Nanumal Bhojraj Restaurant - Fort,Indian Restaurant,"Near GPO, Fort, VT",S. Bhagat Singh Road,18.937974,72.837663,"[{'label': 'display', 'lat': 18.93797448587734...",260,IN,Mumbai,Mahārāshtra,India,"[Near GPO, Fort, VT (S. Bhagat Singh Road), Mu...",400001,5263f0f5498edc1ad5f3270f
2,Iran Like Restaurant,Indian Restaurant,"Palton Road, Near Haj House",NaN,18.945343,72.836809,"[{'label': 'display', 'lat': 18.94534293649397...",747,IN,Mumbai,Mahārāshtra,India,"[Palton Road, Near Haj House, Mumbai 400001, M...",400001,4fb5276be4b0b34b9ac27a1b
3,New Majestic Restaurant,Indian Restaurant,Opp CST,NaN,18.938972,72.835517,"[{'label': 'display', 'lat': 18.93897156089674...",29,IN,Mumbai,Mahārāshtra,India,"[Opp CST, Mumbai, Mahārāshtra, India]",NaN,4d9f2bfe9b91a1cda6c365c0
4,Stadium Restaurant & Stores,Indian Restaurant,Veer Nariman Road,"Next to Vodafone Store, Churchgate",18.933173,72.826929,"[{'label': 'display', 'lat': 18.93317339191467...",1082,IN,Mumbai,Mahārāshtra,India,"[Veer Nariman Road (Next to Vodafone Store, Ch...",400023,4c09449eed2595217aee1256


In [41]:
clean_Restaurant_dataframe2= clean_Restaurant_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'formattedAddress',\
                                        'labeledLatLngs', 'postalCode', 'id','state','distance'], axis=1)
clean_Restaurant_dataframe2 #dropping all the unwanted colummns from the dataframe

,name,categories,address,lat,lng
0,Farhang Restaurant,Steakhouse,Opposite GPO,18.938165,72.837917
1,Nanumal Bhojraj Restaurant - Fort,Indian Restaurant,"Near GPO, Fort, VT",18.937974,72.837663
2,Iran Like Restaurant,Indian Restaurant,"Palton Road, Near Haj House",18.945343,72.836809
3,New Majestic Restaurant,Indian Restaurant,Opp CST,18.938972,72.835517
4,Stadium Restaurant & Stores,Indian Restaurant,Veer Nariman Road,18.933173,72.826929
5,Sai Pooja Restaurant,None,NaN,18.938357,72.835073
6,Panchratna restaurant & bar,Indian Restaurant,Opp. Metro Cinema,18.942900,72.826820
7,Poonam Restaurant,Restaurant,NaN,18.939225,72.835707
8,Nanumal Bhojraj Restaurant - Masjid Bunder,Indian Restaurant,"Gaumukh Bhawan, Masjid Bunder West, Near Masji...",18.953200,72.837577
9,Aram Restaurant,Asian Restaurant,"Capitol building, DN Road",18.939543,72.834495


In [42]:
df_Restaurant = clean_Restaurant_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant #dropping all the null values in the data frame

,name,categories,address,lat,lng
0,Farhang Restaurant,Steakhouse,Opposite GPO,18.938165,72.837917
1,Nanumal Bhojraj Restaurant - Fort,Indian Restaurant,"Near GPO, Fort, VT",18.937974,72.837663
2,Iran Like Restaurant,Indian Restaurant,"Palton Road, Near Haj House",18.945343,72.836809
3,New Majestic Restaurant,Indian Restaurant,Opp CST,18.938972,72.835517
4,Stadium Restaurant & Stores,Indian Restaurant,Veer Nariman Road,18.933173,72.826929
6,Panchratna restaurant & bar,Indian Restaurant,Opp. Metro Cinema,18.942900,72.826820
8,Nanumal Bhojraj Restaurant - Masjid Bunder,Indian Restaurant,"Gaumukh Bhawan, Masjid Bunder West, Near Masji...",18.953200,72.837577
9,Aram Restaurant,Asian Restaurant,"Capitol building, DN Road",18.939543,72.834495
11,Kamat's Restaurant,Indian Restaurant,"Opp. Electric House, Colaba",18.921570,72.831004
14,Sind Punjab Bar & Restaurant,Restaurant,"263, SBS Road, Fort, Mumbai",18.937048,72.837381


Hotels

In [43]:
search_query = 'Hotel'
radius = 1000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=5J5PBAMUUA22S2KM2USBTRLCEWPHIKO0P05HOZA5KEGKLWDZ&client_secret=NKOKFXFBVPRWAUUSSR0F4UVELLHYX4YVALN1HRJHLMUP45LK&ll=18.9387711,72.8353355&v=20180604&query=Hotel&radius=1000&limit=30'

In [44]:
results = requests.get(url).json()

In [45]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe1 = json_normalize(venues)
dataframe1.head()

C:\Users\deepa\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4e135a03a809291902c7d32f,Hotel Traveller’s Inn,"[{'id': '4bf58dd8d48988d1f8931735', 'name': 'B...",v-1588315768,False,"26, ADI Marzban Path","Behind Cafe Universal, Ballard Estate, Fort",18.936039,72.838000,"[{'label': 'display', 'lat': 18.93603902855253...",413,400001,IN,Mumbai,Mahārāshtra,India,"[26, ADI Marzban Path (Behind Cafe Universal, ..."
1,4bc6c48704e8b713f57f362d,Hotel Deluxe,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1588315768,False,Pitha Street,Fort,18.933915,72.833205,"[{'label': 'display', 'lat': 18.93391483135239...",585,400001,IN,Mumbai,Mahārāshtra,India,"[Pitha Street (Fort), Mumbai 400001, Mahārāsht..."
2,4bcf4de40ffdce72215fb2c0,Residency Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1588315768,False,"26, Corner of D.N. Road and Rustom Sidhwa Marg...",NaN,18.934978,72.833481,"[{'label': 'display', 'lat': 18.93497786680689...",465,400001,IN,Mumbai,Mahārāshtra,India,"[26, Corner of D.N. Road and Rustom Sidhwa Mar..."
3,4cbc460f9552b60c8d81e48b,City Palace Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1588315768,False,NaN,NaN,18.938970,72.835610,"[{'label': 'display', 'lat': 18.93897, 'lng': ...",36,NaN,IN,NaN,NaN,India,[India]
4,4ffaf206e4b0a0306dd1271d,Hotel City Palace,[],v-1588315768,False,Near CST,NaN,18.938474,72.834599,"[{'label': 'display', 'lat': 18.93847362379592...",84,NaN,IN,Mumbai,Mahārāshtra,India,"[Near CST, Mumbai, Mahārāshtra, India]"


In [46]:
clean_columns = ['name', 'categories'] + [col for col in dataframe1.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe1.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Hotel Traveller’s Inn,Bed & Breakfast,"26, ADI Marzban Path","Behind Cafe Universal, Ballard Estate, Fort",18.936039,72.838000,"[{'label': 'display', 'lat': 18.93603902855253...",413,400001,IN,Mumbai,Mahārāshtra,India,"[26, ADI Marzban Path (Behind Cafe Universal, ...",4e135a03a809291902c7d32f
1,Hotel Deluxe,Indian Restaurant,Pitha Street,Fort,18.933915,72.833205,"[{'label': 'display', 'lat': 18.93391483135239...",585,400001,IN,Mumbai,Mahārāshtra,India,"[Pitha Street (Fort), Mumbai 400001, Mahārāsht...",4bc6c48704e8b713f57f362d
2,Residency Hotel,Hotel,"26, Corner of D.N. Road and Rustom Sidhwa Marg...",NaN,18.934978,72.833481,"[{'label': 'display', 'lat': 18.93497786680689...",465,400001,IN,Mumbai,Mahārāshtra,India,"[26, Corner of D.N. Road and Rustom Sidhwa Mar...",4bcf4de40ffdce72215fb2c0
3,City Palace Hotel,Hotel,NaN,NaN,18.938970,72.835610,"[{'label': 'display', 'lat': 18.93897, 'lng': ...",36,NaN,IN,NaN,NaN,India,[India],4cbc460f9552b60c8d81e48b
4,Hotel City Palace,None,Near CST,NaN,18.938474,72.834599,"[{'label': 'display', 'lat': 18.93847362379592...",84,NaN,IN,Mumbai,Mahārāshtra,India,"[Near CST, Mumbai, Mahārāshtra, India]",4ffaf206e4b0a0306dd1271d


In [48]:
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','postalCode', 'id','state'], axis=1)
clean_dataframe2

,name,categories,address,lat,lng
0,Hotel Traveller’s Inn,Bed & Breakfast,"26, ADI Marzban Path",18.936039,72.838000
1,Hotel Deluxe,Indian Restaurant,Pitha Street,18.933915,72.833205
2,Residency Hotel,Hotel,"26, Corner of D.N. Road and Rustom Sidhwa Marg...",18.934978,72.833481
3,City Palace Hotel,Hotel,NaN,18.938970,72.835610
4,Hotel City Palace,None,Near CST,18.938474,72.834599
5,sheel hotel,Hotel,NaN,18.938211,72.835725
6,Hotel Fountain Plaza,Indian Restaurant,"panthaky house, Raghunath dadaji street",18.935671,72.834030
7,Hotel Anand Bhuvan,Fast Food Restaurant,Opposite Shahid bhagat Singh Road,18.935345,72.836884
8,hotel kalapi,Motel,NaN,18.938184,72.837802
9,Hotel Landmark Fort,Bed & Breakfast,249 P D Mello Road,18.938528,72.837998


In [49]:
clean_dataframe2.isnull().sum()

name           0
categories     2
address       13
lat            0
lng            0
dtype: int64

In [50]:
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe3

,name,categories,address,lat,lng
0,Hotel Traveller’s Inn,Bed & Breakfast,"26, ADI Marzban Path",18.936039,72.838000
1,Hotel Deluxe,Indian Restaurant,Pitha Street,18.933915,72.833205
2,Residency Hotel,Hotel,"26, Corner of D.N. Road and Rustom Sidhwa Marg...",18.934978,72.833481
6,Hotel Fountain Plaza,Indian Restaurant,"panthaky house, Raghunath dadaji street",18.935671,72.834030
7,Hotel Anand Bhuvan,Fast Food Restaurant,Opposite Shahid bhagat Singh Road,18.935345,72.836884
9,Hotel Landmark Fort,Bed & Breakfast,249 P D Mello Road,18.938528,72.837998
10,Colaba Grand Hotel,Hotel Bar,Colaba,18.939656,72.836457
15,Hotel Windsor,Hostel,Kumptha 10,18.936808,72.838327
18,Hotel Dolphin,Hotel,"327,SBS Road",18.934567,72.836942
19,Welcome Hotel,Motel,257 Shahid Bhagat Singh Rd.,18.937703,72.837467


In [53]:
# delete rows which its category is not Hotel
array= ['Hotel', 'Motel','Bed & Breakfast','Hotel Bar','Hotel Pool']
hotel_dataframe= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)]
hotel_dataframe

,name,categories,address,lat,lng
0,Hotel Traveller’s Inn,Bed & Breakfast,"26, ADI Marzban Path",18.936039,72.838000
2,Residency Hotel,Hotel,"26, Corner of D.N. Road and Rustom Sidhwa Marg...",18.934978,72.833481
9,Hotel Landmark Fort,Bed & Breakfast,249 P D Mello Road,18.938528,72.837998
10,Colaba Grand Hotel,Hotel Bar,Colaba,18.939656,72.836457
18,Hotel Dolphin,Hotel,"327,SBS Road",18.934567,72.836942
19,Welcome Hotel,Motel,257 Shahid Bhagat Singh Rd.,18.937703,72.837467
23,hotel golden palm fort,Hotel Pool,Fort,18.933103,72.834628
26,Hotel New Bengal,Hotel,Doctor D.N. Road,18.945722,72.833936
27,Hotel West End,Hotel,45 New Marine Lines,18.940741,72.827720


Shopping Centres

In [57]:
search_query = 'Shopping'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=5J5PBAMUUA22S2KM2USBTRLCEWPHIKO0P05HOZA5KEGKLWDZ&client_secret=NKOKFXFBVPRWAUUSSR0F4UVELLHYX4YVALN1HRJHLMUP45LK&ll=18.9387711,72.8353355&v=20180604&query=Shopping&radius=10000&limit=30'

In [58]:
sresults = requests.get(url).json()

In [59]:
venues = sresults['response']['venues']

# tranform venues into a dataframe
Shopping_dataframe = json_normalize(venues)
Shopping_dataframe.head()

C:\Users\deepa\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.crossStreet,location.postalCode
0,4e54cc34ae60e3d6b7f18c28,Ashoka Shopping Centre,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1588316409,False,18.945182,72.832121,"[{'label': 'display', 'lat': 18.94518213019515...",789,IN,Mumbai,Mahārāshtra,India,"[Mumbai, Mahārāshtra, India]",NaN,NaN,NaN
1,527b9fd8498e3c08ce24052d,Ashrafi Shopping Centre,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1588316409,False,18.948688,72.836013,"[{'label': 'display', 'lat': 18.9486880047746,...",1106,IN,NaN,NaN,India,[India],NaN,NaN,NaN
2,4ecf3a6ce5faa5ec01659919,Bhangwadi Shopping Arcade,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1588316409,False,18.946791,72.827999,"[{'label': 'display', 'lat': 18.94679133422852...",1180,IN,NaN,NaN,India,[India],NaN,NaN,NaN
3,5aa91198838e594647f44f87,Shopping of Indian Contemporary Arts,"[{'id': '4bf58dd8d48988d1e2931735', 'name': 'A...",v-1588316409,False,18.925923,72.821284,"[{'label': 'display', 'lat': 18.92592268978104...",2057,IN,Mumbai,Mahārāshtra,India,"[Nariman Point (Nariman Point,), Mumbai 400021...",Nariman Point,"Nariman Point,",400021
4,4b0587def964a520dda422e3,Oberoi Shopping Centre,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1588316409,False,18.927008,72.820076,"[{'label': 'display', 'lat': 18.92700822853225...",2072,IN,Mumbai,Mahārāshtra,India,"[Sir Dorab Tata Road, Mumbai 400 021, Mahārāsh...",Sir Dorab Tata Road,NaN,400 021


In [60]:
Shopping_clean_columns = ['name', 'categories'] + [col for col in Shopping_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Shopping_dataframe = Shopping_dataframe.loc[:,Shopping_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list5 = row['categories']
    except:
        categories_list5 = row['venue.categories']
        
    if len(categories_list5) == 0:
        return None
    else:
        return categories_list5[0]['name']

# filter the category for each row
clean_Shopping_dataframe['categories'] = clean_Shopping_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Shopping_dataframe.columns = [column.split('.')[-1] for column in clean_Shopping_dataframe.columns]

clean_Shopping_dataframe.head()

,name,categories,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,address,crossStreet,postalCode,id
0,Ashoka Shopping Centre,Office,18.945182,72.832121,"[{'label': 'display', 'lat': 18.94518213019515...",789,IN,Mumbai,Mahārāshtra,India,"[Mumbai, Mahārāshtra, India]",NaN,NaN,NaN,4e54cc34ae60e3d6b7f18c28
1,Ashrafi Shopping Centre,Office,18.948688,72.836013,"[{'label': 'display', 'lat': 18.9486880047746,...",1106,IN,NaN,NaN,India,[India],NaN,NaN,NaN,527b9fd8498e3c08ce24052d
2,Bhangwadi Shopping Arcade,Office,18.946791,72.827999,"[{'label': 'display', 'lat': 18.94679133422852...",1180,IN,NaN,NaN,India,[India],NaN,NaN,NaN,4ecf3a6ce5faa5ec01659919
3,Shopping of Indian Contemporary Arts,Art Gallery,18.925923,72.821284,"[{'label': 'display', 'lat': 18.92592268978104...",2057,IN,Mumbai,Mahārāshtra,India,"[Nariman Point (Nariman Point,), Mumbai 400021...",Nariman Point,"Nariman Point,",400021,5aa91198838e594647f44f87
4,Oberoi Shopping Centre,Shopping Mall,18.927008,72.820076,"[{'label': 'display', 'lat': 18.92700822853225...",2072,IN,Mumbai,Mahārāshtra,India,"[Sir Dorab Tata Road, Mumbai 400 021, Mahārāsh...",Sir Dorab Tata Road,NaN,400 021,4b0587def964a520dda422e3


In [61]:
clean_Shopping_dataframe2= clean_Shopping_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'postalCode', 'id','state'], axis=1)
clean_Shopping_dataframe2

,name,categories,lat,lng,address
0,Ashoka Shopping Centre,Office,18.945182,72.832121,NaN
1,Ashrafi Shopping Centre,Office,18.948688,72.836013,NaN
2,Bhangwadi Shopping Arcade,Office,18.946791,72.827999,NaN
3,Shopping of Indian Contemporary Arts,Art Gallery,18.925923,72.821284,Nariman Point
4,Oberoi Shopping Centre,Shopping Mall,18.927008,72.820076,Sir Dorab Tata Road
5,Mufaddal Shopping Arcade,Shopping Mall,18.962679,72.836868,NaN
6,Heera Panna Shopping Center,Shopping Mall,18.977096,72.811496,Bhulabhai Desai Road
7,Infinite Online Shopping Pvt. Ltd.,Office,18.998970,72.826539,Lower Parel
8,Sahakar Nagar shopping market,Department Store,19.016399,72.857880,Sahakar Nagar
9,central shopping market,Automotive Shop,19.031664,72.848396,NaN


In [62]:
clean_Shopping_dataframe2.isnull().sum()

name          0
categories    0
lat           0
lng           0
address       5
dtype: int64

In [63]:
clean_dataframe4 = clean_Shopping_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe4

,name,categories,lat,lng,address
3,Shopping of Indian Contemporary Arts,Art Gallery,18.925923,72.821284,Nariman Point
4,Oberoi Shopping Centre,Shopping Mall,18.927008,72.820076,Sir Dorab Tata Road
6,Heera Panna Shopping Center,Shopping Mall,18.977096,72.811496,Bhulabhai Desai Road
7,Infinite Online Shopping Pvt. Ltd.,Office,18.998970,72.826539,Lower Parel
8,Sahakar Nagar shopping market,Department Store,19.016399,72.857880,Sahakar Nagar
10,Palladium,Shopping Mall,18.994317,72.824398,"High Street Phoenix,"
11,JNPT Shopping Center,Shopping Mall,18.876703,72.938897,JNPT


In [65]:
#dataframe of hotels, shopping stores and restaurants
df = pd.concat([hotel_dataframe, df_Restaurant, clean_dataframe4], ignore_index=True)
df

,name,categories,address,lat,lng
0,Hotel Traveller’s Inn,Bed & Breakfast,"26, ADI Marzban Path",18.936039,72.838000
1,Residency Hotel,Hotel,"26, Corner of D.N. Road and Rustom Sidhwa Marg...",18.934978,72.833481
2,Hotel Landmark Fort,Bed & Breakfast,249 P D Mello Road,18.938528,72.837998
3,Colaba Grand Hotel,Hotel Bar,Colaba,18.939656,72.836457
4,Hotel Dolphin,Hotel,"327,SBS Road",18.934567,72.836942
5,Welcome Hotel,Motel,257 Shahid Bhagat Singh Rd.,18.937703,72.837467
6,hotel golden palm fort,Hotel Pool,Fort,18.933103,72.834628
7,Hotel New Bengal,Hotel,Doctor D.N. Road,18.945722,72.833936
8,Hotel West End,Hotel,45 New Marine Lines,18.940741,72.827720
9,Farhang Restaurant,Steakhouse,Opposite GPO,18.938165,72.837917


In [68]:
df_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(df['lat'], df['lng'], df['name'],df['categories'],df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(df_map)  
    
df_map #mapping all of these on a map